In [11]:
!apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [12]:
import os
os.getcwd()

'/teamspace/studios/this_studio'

In [14]:
import os
path = os.getcwd() +'/FinalProject/wav2vec-kenlm/data/mesivta/'
file = path + 'mesivta_cleaned_for_kenlm.txt'

before downloading and unpacking the KenLM repo.

In [15]:
import platform
import os

if platform.system() == 'Darwin':  # Darwin stands for MacOS
    !curl -L https://kheafield.com/code/kenlm.tar.gz | tar xz
elif platform.system() == 'Linux':
    !wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
else:
    print("Unsupported operating system")

--2024-05-30 14:18:19--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  2.96MB/s    in 0.2s    

2024-05-30 14:18:19 (2.96 MB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [16]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

mkdir: cannot create directory ‘kenlm/build’: File exists
build_binary  fragment	       lmplz			     query
count_ngrams  interpolate      phrase_table_vocab	     streaming_example
filter	      kenlm_benchmark  probing_hash_table_benchmark


In [17]:
!pip install pyctcdecode
!pip install kenlm -U


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [18]:
import os

In [19]:
!kenlm/build/bin/lmplz -o 5 <"{file}" > "{path}/5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /teamspace/studios/this_studio/FinalProject/wav2vec-kenlm/data/mesivta/mesivta_cleaned_for_kenlm.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 6190883 types 177867
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:2134404 2:1270403968 3:2382007808 4:3811212032 5:5558018048
Statistics:
1 177867 D1=0.592216 D2=1.069 D3+=1.53844
2 2103408 D1=0.768378 D2=1.1658 D3+=1.45415
3 3946918 D1=0.866782 D2=1.27661 D3+=1.50517
4 4339695 D1=0.925351 D2=1.39269 D3+=1.6002
5 3944388 D1=0.931179 D2=1.43561 D3+=1.57081
Memory estimate for binary LM:
type     MB
probing 309 assuming -p 1.5
probing 370 assuming -r models -p 1.5
trie    155 without quantization
trie     86 assuming -q 8 -b 8 quantization 
trie    136 assuming -a 22 array pointer compression
trie     67 a

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [20]:
!head -20 {path}5gram.arpa

\data\
ngram 1=177867
ngram 2=2103408
ngram 3=3946918
ngram 4=4339695
ngram 5=3944388

\1-grams:
-6.30688	<unk>	0
0	<s>	-1.1736012
-1.4652324	</s>	0
-5.510064	ושננתם	-0.26999253
-5.262256	לבניך	-0.21920805
-5.4479094	ודברת	-0.5241966
-4.5865335	בם	-0.29375523
-5.301768	בשבתך	-0.3935146
-4.9905014	בביתך	-0.2767276
-5.778956	ובלכתך	-0.22316664
-3.585334	בדרך	-0.41843444
-5.582625	ובשכבך	-0.42680454


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [21]:
with open(f"{path}5gram.arpa", "r") as read_file, open(f"{path}5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [22]:
!head -20 {path}5gram_correct.arpa

\data\
ngram 1=177868
ngram 2=2103408
ngram 3=3946918
ngram 4=4339695
ngram 5=3944388

\1-grams:
-6.30688	<unk>	0
0	<s>	-1.1736012
0	</s>	-1.1736012
-1.4652324	</s>	0
-5.510064	ושננתם	-0.26999253
-5.262256	לבניך	-0.21920805
-5.4479094	ודברת	-0.5241966
-4.5865335	בם	-0.29375523
-5.301768	בשבתך	-0.3935146
-4.9905014	בביתך	-0.2767276
-5.778956	ובלכתך	-0.22316664
-3.585334	בדרך	-0.41843444


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## Create the Processor

In [46]:
# from transformers import AutoProcessor
from transformers import AutoProcessor, Wav2Vec2BertProcessor
processor = AutoProcessor.from_pretrained("imvladikon/wav2vec2-xls-r-300m-hebrew",
                                            unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
)
# processor=Wav2Vec2BertProcessor.from_pretrained("/teamspace/studios/this_studio/models/facebook/w2v-bert-2.0-finetuned", 
#                                             unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|"
# )


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
processor.tokenizer

Wav2Vec2CTCTokenizer(name_or_path='imvladikon/wav2vec2-xls-r-300m-hebrew', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': ['<s>', '</s>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	28: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	29: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	30: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	31: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [48]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder.

In [49]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path=f"{path}5gram_correct.arpa",
    
)

Loading the LM will be faster if you build a binary file.
Reading /teamspace/studios/this_studio/FinalProject/wav2vec-kenlm/data/mesivta/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [50]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [28]:
# !sudo apt-get install git-lfs tree

Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class.

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [29]:
# from huggingface_hub import Repository

# repo = Repository(local_dir="FinalProject/models/ken-lm", clone_from="imvladikon/wav2vec2-xls-r-300m-hebrew")

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [30]:
# processor_with_lm.save_pretrained("xls-r-300m-sv")

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [31]:
# !tree -h xls-r-300m-sv/

As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [32]:
# !kenlm/build/bin/build_binary xls-r-300m-sv/language_model/5gram_correct.arpa xls-r-300m-sv/language_model/5gram.bin

Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [33]:
# !rm xls-r-300m-sv/language_model/5gram_correct.arpa && tree -h xls-r-300m-sv/

Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [34]:
# repo.push_to_hub(commit_message="Upload lm-boosted decoder")

In [51]:
import shutil
# model_path = "models/wav2vec2bert-MesivtaLm"
model_path = "models/imvladikon/wav2vec2-hebrew-MesivtaLm"
if os.path.isdir(model_path):
    shutil.rmtree(model_path)
processor_with_lm.save_pretrained(model_path) 

In [ ]:
os.getcwd()

'/teamspace/studios/this_studio/FinalProject/wav2vec-kenlm'